In [ ]:
!pip install -U sagemaker

In [1]:
import sagemaker
import boto3

sm_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sm_session.boto_region_name
account = sm_session.account_id()

# Run Inference on Remote cluster

In [2]:
import time
from sagemaker.debugger import (ProfilerConfig, 
                                FrameworkProfile, 
                                DetailedProfilingConfig, 
                                DataloaderProfilingConfig, 
                                PythonProfilingConfig,
                                PythonProfiler, cProfileTimer)

profiler_config=ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(
        detailed_profiling_config=DetailedProfilingConfig(
            start_step=5, 
            num_steps=1
        ),
        dataloader_profiling_config=DataloaderProfilingConfig(
            start_step=7, 
            num_steps=1
        ),
        python_profiling_config=PythonProfilingConfig(
            start_step=9, 
            num_steps=1, 
            python_profiler=PythonProfiler.CPROFILE, 
            cprofile_timer=cProfileTimer.TOTAL_TIME
        )
    )
)

In [11]:
from sagemaker.pytorch import PyTorch

pytorch_estimator = PyTorch('launcher.py',
                            source_dir='src',
                            instance_type='ml.p3.16xlarge',
                            role=role,
                            session=sm_session,
                            instance_count=2,
                            framework_version='1.6.0',
                            base_job_name="hf-distr-inf-amazon-review-dataset",
                            py_version='py3',
                            hyperparameters = {
                                'train-script': 'batch_inference.py',
                                'inference-batch' : 256,
                                'data-split': 'test',
#                                 'dummy-dataset': True, # if you want to run inference on dummy dataset. Otherwise, we use "amazon_polarity"
                                'wandb-project': "transformers-batch-inference",
                                'wandb-api-key': "<ADD YOUR WANDB API>" #or comment WANDB params
                            },
                            profiler_config=profiler_config,
                           )


pytorch_estimator.fit(wait=False)


# Troubleshoot Locally

In [ ]:
! pip install 'sagemaker[local]' --upgrade

In [4]:
from sagemaker.local import LocalSession

# Configure our local training session
sagemaker_local_session = LocalSession()
sagemaker_local_session.config = {'local': {'local_code': True}}

In [ ]:
from sagemaker.pytorch import PyTorch

pytorch_estimator = PyTorch('launcher.py',
                            source_dir='src',
                            instance_type='local_gpu',
                            role=role,
                            instance_count=1,
                            framework_version='1.6.0',
                            session=sagemaker_local_session,
                            py_version='py3',
                            hyperparameters = {
                                'train-script': 'batch_inference.py',
                                'inference-batch' : 256,
                                'data-split': 'test',
                                'dummy-dataset': True
                            }
                           )


pytorch_estimator.fit()
